Example Profiling
==============================



Example using cProfile/pstats
-----------------------------

Following an example how to profile the algorithm using https://docs.python.org/3/library/profile.html


In [1]:
import IPTestSuite
import IPLazyPRM_Roundtrip
import IPVISLazyPRM

import IPVISVisibilityPRM
import IPVisibilityPRM_Roundtrip

import IPBasicPRM_Roundtrip
import IPVISBasicPRM

import HelperClass
from IPPerfMonitor import IPPerfMonitor

import IPVISBasicPRM



from __future__ import print_function
import sys

from IPython.utils import io

from matplotlib import pylab as plt

benchList = IPTestSuite.benchList
print(benchList)


start = benchList[0].startList
goal = benchList[0].goalList

plannerFactory = dict()

basicConfig = dict()
basicConfig["radius"] = 3
basicConfig["numNodes"] = 200
plannerFactory["basePRM"] = [IPBasicPRM_Roundtrip.BasicPRM, basicConfig, IPVISBasicPRM.basicPRMVisualize]

visbilityConfig = dict()
visbilityConfig["ntry"] = 300
plannerFactory["visibilityPRM"] = [IPVisibilityPRM_Roundtrip.VisPRM, visbilityConfig, IPVISVisibilityPRM.visibilityPRMVisualize ]

lazyConfig = dict()
lazyConfig["initialRoadmapSize"] = 50
lazyConfig["updateRoadmapSize"]  = 2
lazyConfig["kNearest"] = 10
plannerFactory["lazyPRM"] = [IPLazyPRM_Roundtrip.LazyPRM, lazyConfig, IPVISLazyPRM.lazyPRMVisualize]

[<IPBenchmark.Benchmark object at 0x7f54542f2550>, <IPBenchmark.Benchmark object at 0x7f540e92fb50>, <IPBenchmark.Benchmark object at 0x7f540e92ff10>, <IPBenchmark.Benchmark object at 0x7f540e94c6d0>, <IPBenchmark.Benchmark object at 0x7f540e94d350>]


In [2]:
class ResultCollection (object):
    
    def __init__(self, plannerFactoryName, planner, benchmark, solution, perfDataFrame):
        self.plannerFactoryName = plannerFactoryName
        self.planner = planner
        self.benchmark = benchmark
        self.solution = solution
        self.perfDataFrame = perfDataFrame

In [3]:
resultList = list()
testList = benchList[:1]
i = 0
for key,producer in list(plannerFactory.items()):
    HelperClass.HelperClass.printInColor(str(key)+" "+ str(producer), 'yellow')
    #print(key, producer)
    print(key)
    for benchmark in testList:
        #HelperClass.HelperClass.printInColor("Planning: " + key + " - " + benchmark.name, 'yellow')
        #planner = IPBasicPRM.BasicPRM(benchmark.collisionChecker)
        planner = producer[0](benchmark.collisionChecker)
        IPPerfMonitor.clearData()
        
        print("Hallo")
        try:
            with io.capture_output() as captured:
                resultList.append(ResultCollection(key,
                                        planner, 
                                        benchmark, 
                                        planner.planRoundPath(benchmark.startList,benchmark.interimGoalList,benchmark.goalList,producer[1]),
                                        IPPerfMonitor.dataFrame()),)
            #a = planner.planPath(benchmark.startList,benchmark.goalList,producer[1])
            # print("")
            i = i + 1

        except Exception as e:
            HelperClass.HelperClass.printInColor("PLANNING ERROR ! PLANNING ERROR ! PLANNING ERROR", 'red')
            HelperClass.HelperClass.printInColor(e, 'red')
            pass

        

basePRM
Hallo


visibilityPRM
Hallo


lazyPRM
Hallo


In [4]:
# Executing the planning. 
import cProfile
from IPPRMBase import PRMBase

for i in plannerFactory:
    print("i",i)
#print(plannerFactory.)

print("plannerFactory 1", plannerFactory["basePRM"])

plannerBasic = resultList[0].planner
plannerBasicProducer = plannerFactory["basePRM"]
plannerVisibility = resultList[1].planner
plannerVisibilityProducer = plannerFactory["visibilityPRM"]
plannerLazy = resultList[2].planner
plannerLazyProducer = plannerFactory["lazyPRM"]


HelperClass.HelperClass.printInColor("BasicPRMRound", "red")
with io.capture_output() as capturedBasic:
    cProfile.run('solution =plannerBasic.planRoundPath(benchList[0].startList,benchList[0].interimGoalList,benchList[0].goalList,plannerBasicProducer)','basicstats') 

HelperClass.HelperClass.printInColor("VisibiltyPRMRound", "orange")
with io.capture_output() as capturedVisibility:
    cProfile.run('solution = plannerVisibility.planRoundPath(benchList[0].startList,benchList[0].interimGoalList,benchList[0].goalList,plannerVisibilityProducer)','visibilitystats') 
HelperClass.HelperClass.printInColor("LazyPRMRound", "green")
with io.capture_output() as capturedVisibility:
    cProfile.run('solution = plannerLazy.planRoundPath(benchList[0].startList,benchList[0].interimGoalList,benchList[0].goalList,plannerLazyProducer)','lazystats')

i basePRM
i visibilityPRM
i lazyPRM
plannerFactory 1 [<class 'IPBasicPRM_Roundtrip.BasicPRM'>, {'radius': 3, 'numNodes': 200}, <function basicPRMVisualize at 0x7f540cbf2020>]


TypeError: list indices must be integers or slices, not str

In [ ]:
# utility functions to read and access data in astartstats
import pstats
from pstats import SortKey

p = pstats.Stats('visibilitystats')
p.strip_dirs().sort_stats(SortKey.CALLS).print_stats()
p.sort_stats(SortKey.CUMULATIVE).print_stats(15)
p.sort_stats(SortKey.CALLS).print_stats(15)
p.sort_stats(SortKey.TIME).print_stats(15)

fig_local = plt.figure(figsize=(10,10))
ax = fig_local.add_subplot(1,1,1)
  
    
# Anzeige der Benchmarks, noch nicht mit allen Vis möglich

for bench, result in zip(testList, resultList):
    title = bench.name
    pathLength = dict()
    planningTime = dict()
    roadmapSize  = dict()
    if result.benchmark.name == bench.name:
        #print result.benchmark.name  + " - " +  result.plannerFactoryName, len(result.solution)
        pathLength[result.plannerFactoryName] = len(result.solution)
        planningTime[result.plannerFactoryName] = result.perfDataFrame.groupby(["name"]).sum(numeric_only=True)["time"]["planRoundPath"]
        roadmapSize[result.plannerFactoryName] = result.planner.graph.size()

    IPVISVisibilityPRM.visibilityPRMVisualize(result.planner, result.solution, ax=ax, nodeSize=100)

Example using IPerfMonitor
--------------------------

In [ ]:
from IPPerfMonitor import IPPerfMonitor

In [ ]:
benchList = IPTestSuite.benchList

# KOnfig steht in plannerFactory
print(str(planner))
with io.capture_output() as captured:
    solution = planner.planRoundPath(benchList[0].startList,benchList[0].interimGoalList,benchList[0].goalList,producer[1])


start = benchList[0].startList
goal = benchList[0].goalList

data_frame = IPPerfMonitor.dataFrame()

#fig_local = plt.figure(figsize=(10,10))
#ax = fig_local.add_subplot(1,1,1)

#IPVISVisibilityPRM.visibilityPRMVisualize(planner, solution, ax=ax, nodeSize=100)

In [ ]:
resultList = list()
testList = benchList[:1]

solution = list()
data_frame = list()
i = 0
for key,producer in list(plannerFactory.items()):
    HelperClass.HelperClass.printInColor(str(key)+" "+ str(producer), 'yellow')

    for benchmark in testList:

        planner = producer[0](benchmark.collisionChecker)
        IPPerfMonitor.clearData()
        
        try:
            with io.capture_output() as captured:
                #resultList.append(ResultCollection(key,
                #                        planner, 
                #                        benchmark, 
                #                        planner.planRoundPath(benchmark.startList,benchmark.interimGoalList,benchmark.goalList,producer[1]),
                #                        IPPerfMonitor.dataFrame()),)
                solution.append(planner.planRoundPath(benchList[0].startList,benchList[0].interimGoalList,benchList[0].goalList,producer[1]))
                data_frame.append(IPPerfMonitor.dataFrame())

                i = i + 1


        except Exception as e:
            HelperClass.HelperClass.printInColor("PLANNING ERROR ! PLANNING ERROR ! PLANNING ERROR", 'red')
            HelperClass.HelperClass.printInColor(e, 'red')
            pass

print(solution[1])

In [ ]:
#data_frame


In [ ]:
HelperClass.HelperClass.printInColor("Basic", "red")
print(data_frame[0].groupby(["name"]).sum(numeric_only=True))
print("")

HelperClass.HelperClass.printInColor("Visibility", "yellow")
print(data_frame[1].groupby(["name"]).sum(numeric_only=True))
print("")

HelperClass.HelperClass.printInColor("Lazy", "green")
print(data_frame[2].groupby(["name"]).sum(numeric_only=True))

In [ ]:
HelperClass.HelperClass.printInColor("Basic", "red")
print(data_frame[0][data_frame[0]["name"]=="pointInCollision"][data_frame[0]["retVal"]==True].count())

HelperClass.HelperClass.printInColor("Visibility", "yellow")
print(data_frame[1][data_frame[1]["name"]=="pointInCollision"][data_frame[1]["retVal"]==True].count())

HelperClass.HelperClass.printInColor("Lazy", "green")
print(data_frame[2][data_frame[2]["name"]=="pointInCollision"][data_frame[2]["retVal"]==True].count())


In [ ]:
HelperClass.HelperClass.printInColor("Basic", "red")
print(data_frame[0].groupby(["name"]).count())
print("")
HelperClass.HelperClass.printInColor("Visibility", "yellow")
print(data_frame[1].groupby(["name"]).count())
print("")
HelperClass.HelperClass.printInColor("Lazy", "green")
print(data_frame[2].groupby(["name"]).count())



Second Example using IPPerfMonitor
----------------------------------

In [ ]:
import IPVisibilityPRM_Roundtrip
import IPVISVisibilityPRM

In [ ]:
visbilityConfig = dict()
visbilityConfig["ntry"] = 300

vis = plannerFactory["visibilityPRM"][0](benchList[3].collisionChecker)

start = benchList[3].startList
goal = benchList[3].goalList

IPPerfMonitor.clearData()

with io.capture_output() as captured:
    solution = vis.planRoundPath(benchmark.startList,benchmark.interimGoalList,benchmark.goalList,visbilityConfig)

data_frame = IPPerfMonitor.dataFrame()

fig_local = plt.figure(figsize=(10,10))
ax = fig_local.add_subplot(1,1,1)
IPVISVisibilityPRM.visibilityPRMVisualize(vis, solution, ax = ax)

In [ ]:
data_frame

In [ ]:
data_frame.groupby(["name"]).count()

In [ ]:
data_frame.groupby(["name"]).sum(numeric_only=True)